In [1]:
#include "kalman.hpp"

In [2]:
#include <iostream>
#include <stdexcept>
#include <vector>


In [3]:
// helper function: matrix addition
std::vector<std::vector<double>> matadd(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rows = a.size();
    int cols = a[0].size();
    
    std::vector<std::vector<double>> result(rows, std::vector<double>(cols));

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i][j] = a[i][j] + b[i][j];
        }
    }

    return result;
}

In [4]:
// helper function: matrix subtraction
std::vector<std::vector<double>> matsub(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rows = a.size();
    int cols = a[0].size();
    
    std::vector<std::vector<double>> result(rows, std::vector<double>(cols));

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i][j] = a[i][j] - b[i][j];
        }
    }

    return result;
}

In [5]:
// helper function: matrix transpose
std::vector<std::vector<double>> mattranspose(const std::vector<std::vector<double>>& a) {
    int rows = a.size();
    int cols = a[0].size();
    
    std::vector<std::vector<double>> result(cols, std::vector<double>(rows));

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[j][i] = a[i][j];
        }
    }

    return result;
}


In [6]:
// helper function: matrix inversion
std::vector<std::vector<double>> matinverse(const std::vector<std::vector<double>>& a) {
    size_t n = a.size();
    
    if (n != a[0].size()) {
        std::cout<<" Shape of a : "<<a.size()<<","<<a[0].size()<<"\n";
        throw std::runtime_error("Only square matrices are supported for inversion.");
    }

    // Handle 1x1 matrix
    if (n == 1) {
        if (a[0][0] == 0) {
            throw std::runtime_error("Matrix is singular and cannot be inverted.");
        }
        return {{1.0 / a[0][0]}};
    }
    
    if (n == 2) {
        double determinant = a[0][0] * a[1][1] - a[0][1] * a[1][0];
        if (determinant == 0) {
            throw std::runtime_error("Matrix is singular and cannot be inverted.");
        }

        std::vector<std::vector<double>> result(2, std::vector<double>(2));
        result[0][0] = a[1][1] / determinant;
        result[0][1] = -a[0][1] / determinant;
        result[1][0] = -a[1][0] / determinant;
        result[1][1] = a[0][0] / determinant;

        return result;
    }

    if (n == 3) {
        double determinant = a[0][0]*(a[1][1]*a[2][2]-a[2][1]*a[1][2]) 
                             - a[0][1]*(a[1][0]*a[2][2]-a[1][2]*a[2][0]) 
                             + a[0][2]*(a[1][0]*a[2][1]-a[1][1]*a[2][0]);
        
        if (determinant == 0) {
            throw std::runtime_error("Matrix is singular and cannot be inverted.");
        }

        std::vector<std::vector<double>> result(3, std::vector<double>(3));

        result[0][0] = (a[1][1] * a[2][2] - a[2][1] * a[1][2]) / determinant;
        result[0][1] = (a[0][2] * a[2][1] - a[0][1] * a[2][2]) / determinant;
        result[0][2] = (a[0][1] * a[1][2] - a[0][2] * a[1][1]) / determinant;
        result[1][0] = (a[1][2] * a[2][0] - a[1][0] * a[2][2]) / determinant;
        result[1][1] = (a[0][0] * a[2][2] - a[0][2] * a[2][0]) / determinant;
        result[1][2] = (a[1][0] * a[0][2] - a[0][0] * a[1][2]) / determinant;
        result[2][0] = (a[1][0] * a[2][1] - a[2][0] * a[1][1]) / determinant;
        result[2][1] = (a[2][0] * a[0][1] - a[0][0] * a[2][1]) / determinant;
        result[2][2] = (a[0][0] * a[1][1] - a[1][0] * a[0][1]) / determinant;

        return result;
    }

    throw std::runtime_error("Only 2x2 and 3x3 matrices supported for inversion in this function.");
}



In [7]:
// helper function: matrix-vector subtraction
std::vector<std::vector<double>> matvecsub(const std::vector<std::vector<double>>& mat, const std::vector<double>& vec) {
    std::vector<std::vector<double>> result(mat.size(), std::vector<double>(vec.size()));

    for (size_t i = 0; i < mat.size(); i++) {
        for (size_t j = 0; j < vec.size(); j++) {
            result[i][j] = mat[i][j] - vec[j];
        }
    }

    return result;
}

In [8]:
// helper function: matrix-vector multiplication
std::vector<double> matvecmul(const std::vector<std::vector<double>>& a, const std::vector<double>& b) {
    int rows = a.size();
    int cols = a[0].size();

    if (cols != b.size()) {
        throw std::runtime_error("Matrix dimensions do not match for multiplication.");
    }

    std::vector<double> result(rows, 0.0);

    for (int i = 0; i < rows; i++) {
        for (int j = 0; j < cols; j++) {
            result[i] += a[i][j] * b[j];
        }
    }

    return result;
}

In [9]:
// helper function: matrix-matrix multiplication
std::vector<std::vector<double>> matmul(const std::vector<std::vector<double>>& a, const std::vector<std::vector<double>>& b) {
    int rowsA = a.size();
    int colsA = a[0].size();
    int rowsB = b.size();
    int colsB = b[0].size();

    if (colsA != rowsB) {
        throw std::runtime_error("Matrix dimensions do not match for multiplication.");
    }

    std::vector<std::vector<double>> result(rowsA, std::vector<double>(colsB, 0));

    for (int i = 0; i < rowsA; i++) {
        for (int j = 0; j < colsB; j++) {
            for (int k = 0; k < colsA; k++) {
                result[i][j] += a[i][k] * b[k][j];
            }
        }
    }

    return result;
}

In [10]:
// helper function: vector subtraction
std::vector<double> vecsub(const std::vector<double>& a, const std::vector<double>& b) {
    size_t len = a.size();
    std::vector<double> result(len);
    for (size_t i = 0; i < len; i++) {
        result[i] = a[i] - b[i];
    }
    return result;
}

In [11]:
//set up class and constructor
KalmanFilter::KalmanFilter(
    double dt,
    const std::vector<std::vector<double>>& A,
    const std::vector<std::vector<double>>& C,
    const std::vector<std::vector<double>>& Q,
    const std::vector<std::vector<double>>& R,
    const std::vector<std::vector<double>>& P)
  : A(A), C(C), Q(Q), R(R), P0(P),
    m(C.size()), n(A.size()), dt(dt), initialized(false),
    I(n, std::vector<double>(n)), x_hat(n), x_hat_new(n)
{
    for (int i = 0; i < n; i++) {
        I[i][i] = 1.0;
    }
}

KalmanFilter::KalmanFilter() {}

In [12]:
// init the kf states + measurements 
void KalmanFilter::init(double t0, const std::vector<double>& x0) {
    x_hat = x0;
    P = P0;
    this->t0 = t0;
    t = t0;
    initialized = true;
}

In [13]:
void KalmanFilter::init() {
    std::fill(x_hat.begin(), x_hat.end(), 0.0);
    P = P0;
    t0 = 0;
    t = t0;
    initialized = true;
}

In [14]:


void KalmanFilter::update(const std::vector<double>& y) {
    if (!initialized)
        throw std::runtime_error("Filter is not initialized!");

    x_hat_new = matvecmul(A, x_hat);

    P = matadd(matmul(matmul(A, P), mattranspose(A)), Q);

    std::vector<std::vector<double>> inv = matinverse(matadd(matmul(matmul(C, P), mattranspose(C)), R));

    K = matmul(matmul(P, mattranspose(C)), inv);
    std::vector<double> temp = matvecmul(C, x_hat_new);
    std::vector<double> difference = vecsub(y, temp);

    for (size_t i = 0; i < x_hat_new.size(); i++) {
        x_hat_new[i] += matvecmul(K, difference)[i];
    }

    P = matmul(matsub(I, matmul(K, C)), P);

    x_hat = x_hat_new;
    t += dt;
}


In [15]:
void KalmanFilter::update(const std::vector<double>& y, double dt, const std::vector<std::vector<double>>& A) {
    this->A = A;
    this->dt = dt;
    update(y);
}


In [16]:
std::vector<double> generateExpandedData(const std::vector<double>& baseData, int repetitions) {
    std::vector<double> expandedData;
    for (int r = 0; r < repetitions; r++) {
        for (auto val : baseData) {
            // Add slight random perturbation to the data.
            double perturbedValue = val + ((double)rand() / RAND_MAX - 0.5);
            expandedData.push_back(perturbedValue);
        }
    }
    return expandedData;
}

In [17]:
int run_kf(int expansion_factor, bool verbose) {
    
    int n = 3; // Number of states
    int m = 1; // Number of measurements

    double dt = 1.0 / 30; // Time step
    
    std::vector<std::vector<double>> A(n, std::vector<double>(n));
    std::vector<std::vector<double>> C(m, std::vector<double>(n));
    std::vector<std::vector<double>> Q(n, std::vector<double>(n));
    std::vector<std::vector<double>> R(m, std::vector<double>(m));
    std::vector<std::vector<double>> P(n, std::vector<double>(n));
    
    A = {{1, dt, 0}, {0, 1, dt}, {0, 0, 1}};
    C = {{1, 0, 0}};
    Q = {{.05, .05, .0}, {.05, .05, .0}, {.0, .0, .0}};
    R = {{5}};
    P = {{.1, .1, .1}, {.1, 10000, 10}, {.1, 10, 100}};
    
    KalmanFilter kf(dt, A, C, Q, R, P);
    
    std::vector<double> measurements = {
      1.04202710058, 1.10726790452, 1.2913511148, 1.48485250951, 1.72825901034,
      1.74216489744, 2.11672039768, 2.14529225112, 2.16029641405, 2.21269371128,
      2.57709350237, 2.6682215744, 2.51641839428, 2.76034056782, 2.88131780617,
      2.88373786518, 2.9448468727, 2.82866600131, 3.0006601946, 3.12920591669,
      2.858361783, 2.83808170354, 2.68975330958, 2.66533185589, 2.81613499531,
      2.81003612051, 2.88321849354, 2.69789264832, 2.4342229249, 2.23464791825,
      2.30278776224, 2.02069770395, 1.94393985809, 1.82498398739, 1.52526230354,
      1.86967808173, 1.18073207847, 1.10729605087, 0.916168349913, 0.678547664519,
      0.562381751596, 0.355468474885, -0.155607486619, -0.287198661013, -0.602973173813
      };
    
    std::vector<double> expandedMeasurements = generateExpandedData(measurements, expansion_factor);
    
    std::vector<double> x0 = {expandedMeasurements[0], 0, -9.81};
    kf.init(0, x0);

    // Feed measurements into filter, output estimated states
    std::vector<double> y(m);
    if(verbose) {
        std::cout << "t = " << 0 << ", " << "x_hat[0]: ";
        for (auto& val : kf.state()) std::cout << val << " ";
        std::cout << std::endl;
    }
    
    int i;
    for (i = 0; i < measurements.size(); i++) {
        y[0] = measurements[i];
        kf.update(y);
        if(verbose) {
            std::cout << "t = " << (i + 1) * dt << ", y[" << i << "] = " << y[0] << ", x_hat[" << i << "] = ";
            for (auto& val : kf.state()) std::cout << val << " ";
            std::cout << std::endl;
        }
    }
    std::cout << std::endl;
    std::cout<<"Exec Success, Final kf states:";
    for (auto& val : kf.state()) std::cout << val << " ";
    std::cout << std::endl;
    return 0;
}

In [18]:
#include <chrono>
auto start = std::chrono::high_resolution_clock::now();
auto stop = std::chrono::high_resolution_clock::now();
auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(stop - start);

In [19]:
int pyrun_sim(int exp_factor, bool verbose) {
    start = std::chrono::high_resolution_clock::now();
    run_kf(exp_factor, verbose);
    stop = std::chrono::high_resolution_clock::now();
    duration = std::chrono::duration_cast<std::chrono::milliseconds>(stop - start);
    return duration.count();
}

In [20]:
pyrun_sim(10, true)

t = 0, x_hat[0]: 1.49273 0 -9.81 
t = 0.0333333, y[0] = 1.04203, x_hat[0] = 1.18055 -9.56663 -9.82201 
t = 0.0666667, y[1] = 1.10727, x_hat[1] = 1.04216 -7.18534 -9.81834 
t = 0.1, y[2] = 1.29135, x_hat[2] = 1.10856 -4.4342 -9.80991 
t = 0.133333, y[3] = 1.48485, x_hat[3] = 1.23897 -2.65176 -9.79492 
t = 0.166667, y[4] = 1.72826, x_hat[4] = 1.41541 -1.36441 -9.7679 
t = 0.2, y[5] = 1.74216, x_hat[5] = 1.5201 -0.923214 -9.74154 
t = 0.233333, y[6] = 2.11672, x_hat[6] = 1.71571 -0.249681 -9.67866 
t = 0.266667, y[7] = 2.14529, x_hat[7] = 1.85071 -0.0135839 -9.61971 
t = 0.3, y[8] = 2.1603, x_hat[8] = 1.94342 -0.0067291 -9.56589 
t = 0.333333, y[9] = 2.21269, x_hat[9] = 2.01834 -0.0830709 -9.50747 
t = 0.366667, y[10] = 2.57709, x_hat[10] = 2.16226 0.0419556 -9.35945 
t = 0.4, y[11] = 2.66822, x_hat[11] = 2.28825 0.0859778 -9.20125 
t = 0.433333, y[12] = 2.51642, x_hat[12] = 2.34414 -0.0748605 -9.11883 
t = 0.466667, y[13] = 2.76034, x_hat[13] = 2.43631 -0.124624 -8.94316 
t = 0.5, y[14] 

In [23]:
%%python

expand_factor = [500000, 1000000, 10000000, 20000000, 40000000]
benchmarks = []

for i in expand_factor:
    time_kf = cppyy.gbl.pyrun_sim(int(i), 0)
    benchmarks.append(time_kf)



Exec Success, Final kf states:-0.454554 -7.75636 -8.59785 

Exec Success, Final kf states:-0.505705 -8.13916 -9.30595 

Exec Success, Final kf states:-0.488794 -8.0126 -9.07184 

Exec Success, Final kf states:-0.536246 -8.36772 -9.72873 

Exec Success, Final kf states:-0.560802 -8.55149 -10.0687 


In [24]:
%%python

print(benchmarks)

[706, 1404, 13545, 27020, 53801]
